# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, getSlewsFromEventList
from lsst.summit.utils.efdUtils import (makeEfdClient,
                                        getEfdData,
                                        getMostRecentRowWithDataBefore,
                                        efdTimestampToAstropy,
                                       )

In [ ]:
FATABLE = M1M3FATable.FATABLE

In [ ]:
client = makeEfdClient()
dayObs = 20230710
#dayObs = 20230601
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

In [ ]:
az_slews = []
for slew in slews:
    #print(slew.begin, slew.end, slew.duration)
    dT = 30.0
    start = slew.begin-TimeDelta(dT, format='sec')
    end = slew.end+TimeDelta(dT, format='sec')
    hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
    ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
    az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
    el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
    azs = az['actualPosition'].values
    dAz = azs[-1] - azs[0]
    els = el['actualPosition'].values
    dEl = els[-1] - els[0]
    if dEl < 1E-5 and dAz > 100.0:
        print(slew.begin, slew.end, dAz, dEl, els[0])
        az_slews.append(slew)


In [ ]:
slew = az_slews[3]
dT = 30.0
start = slew.begin-TimeDelta(dT, format='sec')
end = slew.end+TimeDelta(dT, format='sec')
#hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
azs = az['actualPosition'].values
dAz = azs[-1] - azs[0]
els = el['actualPosition'].values
dEl = els[-1] - els[0]
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
print(start.isot, end.isot, timestamp)

In [ ]:
%matplotlib inline
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position changes during slew: {timestamp}\nDeltaAz = {dAz:.3f}, DeltaEl = {dEl:.3f}", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        
        smoothed_ims_data = ims_data[plots[plot_counter]].rolling(10).mean()
        smoothed_ims_data = smoothed_ims_data.dropna()
        smoothed_ims_data -= smoothed_ims_data[0]
        
        if plot_counter > 2:
            smoothed_ims_data *= 1E6
            unit = 'microDeg'
        else:
            smoothed_ims_data *= 1E6
            unit = 'um'
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'IMS ({unit})')
        initial_pos = np.median(smoothed_ims_data.values[0:100])
        final_pos = np.median(smoothed_ims_data.values[-100:-1])
        pos_delta = initial_pos - final_pos
        axs[i][j].set_title(plots[plot_counter]+f"\n Position Delta = {pos_delta:.1f} {unit}")

        smoothed_ims_data.plot(ax=axs[i][j], color='red', label='IMS')
        axs[i][j].plot([smoothed_ims_data.index[0], smoothed_ims_data.index[-1]],\
                [limits[plot_counter], limits[plot_counter]], ls='--', color='k')
        axs[i][j].plot([smoothed_ims_data.index[0], smoothed_ims_data.index[-1]],\
                [-limits[plot_counter], -limits[plot_counter]], ls='--', color='k')
        plot_counter += 1
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Data_{timestamp}.png")